In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

# The generated features:
# FamilySize - the size of the passenger's family
# Important-based on the name, whether or not the passenger has an important title
# RealCabin-the actual cabin letter
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch']
train_data['Important'] = train_data['Name'].str.contains('(Master.)|(Major.)|(Rev.)|(Dr.)|(Col.)|(Capt.)|(Countess.)|(Jonkheer.)')
train_data['RealCabin'] = train_data['Cabin'].str[0]
train_data['RealCabin'].fillna('X', inplace=True)
train_data['Age'].fillna(30.0, inplace=True)
train_data['Fare'].fillna(32.2, inplace=True)
features = ['Pclass', 'Important', 'Sex', 'Age', 'FamilySize', 'Fare', 'RealCabin', 'Embarked']

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch']
test_data['Important'] = test_data['Name'].str.contains('(Master.)|(Major.)|(Rev.)|(Dr.)|(Col.)|(Capt.)|(Countess.)|(Jonkheer.)')
test_data['RealCabin'] = test_data['Cabin'].str[0]
test_data['RealCabin'].fillna('X', inplace=True)
test_data['Age'].fillna(30.0, inplace=True)
test_data['Fare'].fillna(32.2, inplace=True)
features = ['Pclass', 'Important', 'Sex', 'Age', 'FamilySize', 'Fare', 'RealCabin', 'Embarked']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

y = train_data["Survived"]

features = ["Pclass", "Important", "Sex", "Age", "FamilySize", "Fare", "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

X_learn, X_valid, y_learn, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 1)
for j in range(1,11):
    model = RandomForestClassifier(n_estimators=100, max_depth=j, random_state=1)
    model.fit(X_learn, y_learn)
    predictions = model.predict(X_valid)
    y_valid_numpy = y_valid.to_numpy()
    sum = 0
    for i in range(len(predictions)):
        if predictions[i] == y_valid_numpy[i]:
            sum += 1
    print(j, " Random forest ", sum/len(predictions))

for j in range(1,10):
    model = KNeighborsClassifier(n_neighbors=j)
    model.fit(X_learn, y_learn)
    predictions = model.predict(X_valid)
    y_valid_numpy = y_valid.to_numpy()
    sum = 0
    for i in range(len(predictions)):
        if predictions[i] == y_valid_numpy[i]:
            sum += 1
    print(j, " NN ", sum/len(predictions))

model = svm.SVC(kernel='linear')
model.fit(X_learn, y_learn)
predictions = model.predict(X_valid)
y_valid_numpy = y_valid.to_numpy()
sum = 0
for i in range(len(predictions)):
    if predictions[i] == y_valid_numpy[i]:
        sum += 1
print("SVM ", sum/len(predictions))

model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")